---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [4]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [7]:
def answer_one():
    
    spam_count = len(spam_data['target'].where(spam_data['target'] == 1).dropna())
    total = len(spam_data)
    
    return (spam_count/total)*100

In [8]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    
    # Create vect
    vectorizer = CountVectorizer()
    # Fit vect to X_train data and transform X_train to X
    X = vectorizer.fit_transform(X_train)
    # Method is replaced by get_feature_names_out() but extracts feature names from vectorizer
    a = vectorizer.get_feature_names()
    longest = max(a, key=len)
    
    return longest

In [35]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [89]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    # Create vect, fit, and transform
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(X_train)
    # Create multinomial bayes classifier with alpha = 0.1 and fit to training data
    clf = MultinomialNB(alpha=0.1).fit(X, y_train)
    # Transform test data using already fit vectorizer (ensures X_train_vec and X_test_vec have same num columns)
    X_test_vec = vectorizer.transform(X_test)
    # Get y_predictions for X_test
    y_pred = clf.predict(X_test_vec)
    # Gen roc_score for true y_test labels and y_pred labels
    roc_score = roc_auc_score(y_test, y_pred)
    
    return roc_score

In [90]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [190]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def answer_four():
    
    # Create vect, fit to X_train and transform to X_train_vec
    vect = TfidfVectorizer()
    X_train_vec = vect.fit_transform(X_train)
    # Get feature names from vect
    feature_names = np.array(vect.get_feature_names())
    # Max(0) seems to reduce the sparse numpy array to one row and same columns
    # Convert to array with tfidf scores, remove one level of list with [0]
    # argsort() returns a list of indices that would sort the array
    sorted_tfidf_index = X_train_vec.max(0).toarray()[0].argsort()
    
    # The smallest feature indices are at the start of the sorted_tfidf_index, largest at the end
    smallest_tfidf_feature_names = feature_names[sorted_tfidf_index[:20]]
    largest_tfidf_feature_names = feature_names[sorted_tfidf_index[:-21:-1]]
    
    smallest_tfidf_values = X_train_vec.max(0).toarray()[0][sorted_tfidf_index[:20]]
    largest_tfidf_values = X_train_vec.max(0).toarray()[0][sorted_tfidf_index[:-21:-1]]
    
    # Convert to data series
    small_series = pd.Series(smallest_tfidf_values, index=smallest_tfidf_feature_names, name=None)
    large_series = pd.Series(largest_tfidf_values, index=largest_tfidf_feature_names, name=None)
    
    # Necessary for autograder apparently
    small_series.index.name = None
    large_series.index.name = None
    
    # Sort the series first on values then on index using np.lexsort, where last column has highest priority
    # '-' indicates reverse order
    small_series = small_series[np.lexsort((small_series.index, small_series.values))]
    large_series = large_series[np.lexsort((large_series.index, -large_series.values))]
    
    return (small_series, large_series)

In [191]:
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [93]:
def answer_five():
    
    # Create vect with min doc frequency = 3, train and transform
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vec = vect.transform(X_train)
    
    # Create classifier with alpha = 0.1 and fit to train data
    clf = MultinomialNB(alpha=0.1).fit(X_train_vec, y_train)
    
    # Transform test data and gen predictions, and compute auc_score
    X_test_vec = vect.transform(X_test)
    y_pred = clf.predict(X_test_vec)
    
    auc_score = roc_auc_score(y_test, y_pred)
    
    return auc_score

In [94]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [103]:
def answer_six():
    
    # Get dfs for spam and not_spam 
    not_spam = spam_data.where(spam_data['target'] == 0).dropna()
    spam = spam_data.where(spam_data['target'] == 1).dropna()
    
    # Create lists of all the text
    not_spam = list(not_spam['text'])
    spam = list(spam['text'])
    
    # Find no entries in list i.e. total entries of spam and not_spam
    not_spam_entries = len(not_spam)
    spam_entries = len(spam)
    
    # Find total number of characters for spam and not spam messages by iterating over each message
    total_chars_not_spam = 0
    for message in not_spam:
        total_chars_not_spam += len(list(message))
        
    total_chars_spam = 0
    for message in spam:
        total_chars_spam += len(list(message))
        
    avg_len_not_spam = total_chars_not_spam/not_spam_entries
    avg_len_spam = total_chars_spam/spam_entries
    
    return (avg_len_not_spam, avg_len_spam)

In [104]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [105]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [135]:
from sklearn.svm import SVC

def answer_seven():
    
    # Create vect with min doc frequency of 5, fit and transform data 
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vec = vect.transform(X_train)
    
    # Create new feature for X_train_vec, length of document
    add_feat = np.array([])
    for row in X_train:
        add_feat = np.append(add_feat, len(row))
    
    # Add feature to X_train_vec
    X_train_vec = add_feature(X_train_vec, add_feat)
    
    # Create vectorized X_test
    X_test_vec = vect.transform(X_test)
    
    # Create new feature for X_test_vec, as above
    add_feat = np.array([])
    for row in X_test:
        add_feat = np.append(add_feat, len(row))
    
    # Add feature to X_test_vec
    X_test_vec = add_feature(X_test_vec, add_feat)
    
    # Create and train SVC classifier
    clf = SVC(C=10000).fit(X_train_vec, y_train)
    # Generate prediction labels
    y_pred = clf.predict(X_test_vec)
    # Compute auc_score
    roc_score = roc_auc_score(y_test, y_pred)
    
    
    return roc_score

In [136]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [137]:
def answer_eight():
    
    # Get dfs for spam and not spam
    not_spam = spam_data.where(spam_data['target'] == 0).dropna()
    spam = spam_data.where(spam_data['target'] == 1).dropna()
    
    # Create lists of both
    not_spam = list(not_spam['text'])
    spam = list(spam['text'])
    # Find total entries in both
    not_spam_entries = len(not_spam)
    spam_entries = len(spam)
    
    # Count total characters that are digits for spam and not spam
    total_chars_not_spam = 0
    for message in not_spam:
        for char in list(message):
            if char.isdigit():
                total_chars_not_spam += 1
        
    total_chars_spam = 0
    for message in spam:
        for char in list(message):
            if char.isdigit():
                total_chars_spam += 1
        
    avg_len_not_spam = total_chars_not_spam/not_spam_entries
    avg_len_spam = total_chars_spam/spam_entries
    
    return (avg_len_not_spam, avg_len_spam)

In [138]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [139]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    
    # Create tfidf vect with min doc frequency of 5 and word ngram range between 1 and 3 (inclusive), fit, transform
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    X_train_vec = vect.transform(X_train)
    
    # Add len of doc features and num digits per doc features to X_train_vec and X_test_vec
    add_feat = np.array([])
    for row in X_train:
        add_feat = np.append(add_feat, len(row))
        
    add_feat2 = np.array([])
    for row in X_train:
        count = 0
        for char in list(row):
            if char.isdigit():
                count += 1
        add_feat2 = np.append(add_feat2, count)
    
    X_train_vec = add_feature(X_train_vec, add_feat)
    X_train_vec = add_feature(X_train_vec, add_feat2)
    
    X_test_vec = vect.transform(X_test)
    
    add_feat = np.array([])
    for row in X_test:
        add_feat = np.append(add_feat, len(row))
        
    add_feat2 = np.array([])
    for row in X_test:
        count = 0
        for char in list(row):
            if char.isdigit():
                count += 1
        add_feat2 = np.append(add_feat2, count)
        
    X_test_vec = add_feature(X_test_vec, add_feat)
    X_test_vec = add_feature(X_test_vec, add_feat2)
    
    # Train logistic regression model, predict test labels and compute auc_score
    clf = LogisticRegression(C=100).fit(X_train_vec, y_train)
    y_pred = clf.predict(X_test_vec)
    
    auc_score = roc_auc_score(y_test, y_pred)
    
    return auc_score

In [140]:
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [145]:
def answer_ten():
    
    # Create not_spam and spam df
    not_spam = spam_data.where(spam_data['target'] == 0).dropna()
    spam = spam_data.where(spam_data['target'] == 1).dropna()
    
    # Make them lists
    not_spam = list(not_spam['text'])
    spam = list(spam['text'])
    # Get total entries for each
    not_spam_entries = len(not_spam)
    spam_entries = len(spam)
    # Count total characters across all messages that aren't alphanumeric or underscores for spam and not spam
    total_chars_not_spam = 0
    for message in not_spam:
        for char in list(message):
            if not char.isalnum() and not char == '_':
                total_chars_not_spam += 1
        
    total_chars_spam = 0
    for message in spam:
        for char in list(message):
            if not char.isalnum() and not char == '_':
                total_chars_spam += 1
        
    avg_len_not_spam = total_chars_not_spam/not_spam_entries
    avg_len_spam = total_chars_spam/spam_entries
    
    return (avg_len_not_spam, avg_len_spam)

In [146]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [186]:
def answer_eleven():
    
    # Create CountVectorizer vect with min doc frequency=5, and *char* ngram range of 2 and 5 inc. train and transform
    vect = CountVectorizer(min_df=5, analyzer='char_wb', ngram_range=(2,5)).fit(X_train)
    X_train_vec = vect.transform(X_train)
    
    # Get feature names from vect and add our new features
    feature_names = np.array(vect.get_feature_names())
    features_to_add = ['length_of_doc', 'digit_count', 'non_word_char_count']
    for feature in features_to_add:
        feature_names = np.append(feature_names, feature)
    
    # Add features length of doc, num digits per doc, and num non-word characters per doc for X_train_vec, X_test_vec
    add_feat = np.array([])
    for row in X_train:
        add_feat = np.append(add_feat, len(row))
        
    add_feat2 = np.array([])
    for row in X_train:
        count = 0
        for char in list(row):
            if char.isdigit():
                count += 1
        add_feat2 = np.append(add_feat2, count)
        
    add_feat3 = np.array([])
    for row in X_train:
        count = 0
        for char in list(row):
            if not char.isalnum() and not char == '_':
                count += 1
        add_feat3 = np.append(add_feat3, count)
    
    X_train_vec = add_feature(X_train_vec, add_feat)
    X_train_vec = add_feature(X_train_vec, add_feat2)
    X_train_vec = add_feature(X_train_vec, add_feat3)
    
    X_test_vec = vect.transform(X_test)
    
    add_feat = np.array([])
    for row in X_test:
        add_feat = np.append(add_feat, len(row))
        
    add_feat2 = np.array([])
    for row in X_test:
        count = 0
        for char in list(row):
            if char.isdigit():
                count += 1
        add_feat2 = np.append(add_feat2, count)
        
    add_feat3 = np.array([])
    for row in X_test:
        count = 0
        for char in list(row):
            if not char.isalnum() and not char == '_':
                count += 1
        add_feat3 = np.append(add_feat3, count)
        
    X_test_vec = add_feature(X_test_vec, add_feat)
    X_test_vec = add_feature(X_test_vec, add_feat2)
    X_test_vec = add_feature(X_test_vec, add_feat3)
    
    # Train logistic regression model, predict X_test labels and compute auc_score
    clf = LogisticRegression(C=100).fit(X_train_vec, y_train)
    y_pred = clf.predict(X_test_vec)
    
    auc_score = roc_auc_score(y_test, y_pred)
    
    # Create list of coefficient index sorted by value, smallest at front, largest at back
    sorted_coef_index = clf.coef_[0].argsort()
    
    # Get 10 smallest and 10 largest coefficient values and names
    smallest_coef_values = clf.coef_[0][sorted_coef_index[0:10]]
    largest_coef_values = clf.coef_[0][sorted_coef_index[:-11:-1]]
    
    smallest_coef_names = feature_names[sorted_coef_index[0:10]]
    largest_coef_names = feature_names[sorted_coef_index[:-11:-1]]
    
    return (auc_score, smallest_coef_names, largest_coef_names)

In [187]:
answer_eleven()

(0.97885931107074342,
 array(['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'], 
       dtype='<U19'),
 array(['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww',
        'ar'], 
       dtype='<U19'))